In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import requests
import io

## read in data. Columns reduced in Excel

In [2]:
guns = pd.read_csv("gun_violence_csv.csv")
#from https://www.kaggle.com/jameslko/gun-violence-data


## narrow to NY State

In [5]:
guns_NY = guns[guns.state == 'New York']


In [8]:
guns_NY.city_or_county.unique()

array(['Mohawk', 'Brooklyn', 'New York (Manhattan)', 'Jamaica',
       'Locust Valley', 'New York', 'Buffalo', 'Rochester', 'Medford',
       'Queens', 'Syracuse', 'Bronx', 'Poughkeepsie', 'Staten Island',
       'Niagara Falls', 'Manhattan', 'Nesconset', 'Troy', 'Schenectady',
       'Monroe County', 'Mendon', 'Albany', 'Colonie', 'Newburgh',
       'Norwich', 'Lake Ronkonkoma', 'Shirley',
       'Ronkonkoma (Lake Ronkonkoma)', 'Lockport', 'Little Valley',
       'Amherst', 'Freedom', 'Yonkers', 'Great Kills', 'Newfane',
       'Wheatfield', 'Bellport', 'Bethlehem', 'Clay', 'Philadelphia',
       'Marlborough', 'Deer Park', 'Onondaga County', 'Beekman',
       'Ellenville', 'Utica', 'Uniondale', 'Endicott', 'Ballston Spa',
       'Melville', 'Pittsford', 'Limestone', 'Highland',
       'South New Berlin', 'Cheektowaga', 'North Tonawanda',
       'Port Jervis', 'Monticello', 'East Amherst', 'Valley Stream',
       'Bath', 'Beacon', 'Kingston', 'Rochester (Irondequoit)',
       'Brockpo

## Convert Lon Lat to coordiantes

In [88]:
guns_NY['Coordinates'] = list(zip(guns_NY.longitude, guns_NY.latitude))

guns_NY['Coordinates'] = guns_NY['Coordinates'].apply(Point)

guns_NY = gpd.GeoDataFrame(guns_NY, geometry='Coordinates')

/Users/mariumsultan/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/mariumsultan/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


## bring in shapefile with all US ZipCode Polygons

In [89]:
zip_codes = gpd.GeoDataFrame.from_file('us_zip/tl_2017_us_zcta510.shp')

In [90]:
zip_codes.head()

,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,43451,43451,B5,G6350,S,63411475,157689,+41.3183010,-083.6174935,"POLYGON ((-83.708733 41.327326, -83.708147 41...."
1,43452,43452,B5,G6350,S,121783674,13437380,+41.5157923,-082.9809454,"POLYGON ((-83.086978 41.537796, -83.0825629999..."
2,43456,43456,B5,G6350,S,9389362,999166,+41.6468445,-082.8226641,"(POLYGON ((-82.835577 41.710823, -82.83515 41...."
3,43457,43457,B5,G6350,S,48035540,0,+41.2673266,-083.4274645,"POLYGON ((-83.49650299999999 41.253708, -83.48..."
4,43458,43458,B5,G6350,S,2573816,39915,+41.5304461,-083.2133648,"POLYGON ((-83.222292 41.531025, -83.2222819999..."


## Check CRS and match crs of both datasets

In [91]:
zip_codes.crs

{u'init': u'epsg:4269'}

In [92]:
guns_NY.crs = {'init' :'epsg:2263'}


In [93]:
guns_NY.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,latitude,longitude,Coordinates
47,483872,4/7/13,New York,New York (Manhattan),900 Garden Way,1,3,40.9018,-73.8511,POINT (-73.8511 40.9018)
7205,109459,2/23/14,New York,New York (Manhattan),NaN,1,0,40.7495,-73.9893,POINT (-73.9893 40.7495)
19487,141757,5/31/14,New York,New York (Manhattan),West 129th Street and St. Nicholas Terrace,0,1,40.8135,-73.9513,POINT (-73.9513 40.8135)
29965,177516,8/4/14,New York,New York (Manhattan),NaN,0,0,40.7495,-73.9893,POINT (-73.9893 40.7495)
36575,190037,9/12/14,New York,New York (Manhattan),Amsterdam Avenue and 82nd Street,0,2,40.7851,-73.9769,POINT (-73.9769 40.7851)


In [94]:
zip_codes.head()

,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,43451,43451,B5,G6350,S,63411475,157689,+41.3183010,-083.6174935,"POLYGON ((-83.708733 41.327326, -83.708147 41...."
1,43452,43452,B5,G6350,S,121783674,13437380,+41.5157923,-082.9809454,"POLYGON ((-83.086978 41.537796, -83.0825629999..."
2,43456,43456,B5,G6350,S,9389362,999166,+41.6468445,-082.8226641,"(POLYGON ((-82.835577 41.710823, -82.83515 41...."
3,43457,43457,B5,G6350,S,48035540,0,+41.2673266,-083.4274645,"POLYGON ((-83.49650299999999 41.253708, -83.48..."
4,43458,43458,B5,G6350,S,2573816,39915,+41.5304461,-083.2133648,"POLYGON ((-83.222292 41.531025, -83.2222819999..."


## spatial join

In [95]:
guns_NYC_zip = gpd.sjoin(zip_codes, guns_NY, how="inner", op='intersects')

In [96]:
guns_NYC_zip.head()

,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,index_right,incident_id,date,state,city_or_county,address,n_killed,n_injured,latitude,longitude
10092,10065,10065,B5,G6350,S,984659,0,+40.7646284,-073.9631438,"POLYGON ((-73.972099 40.765518, -73.971648 40....",88296,409348,9/5/15,New York,New York (Manhattan),First Avenue,0,1,40.7634,-73.9592
10092,10065,10065,B5,G6350,S,984659,0,+40.7646284,-073.9631438,"POLYGON ((-73.972099 40.765518, -73.971648 40....",118734,531342,3/29/16,New York,New York (Manhattan),2nd Avenue,0,1,40.7644,-73.9616
10092,10065,10065,B5,G6350,S,984659,0,+40.7646284,-073.9631438,"POLYGON ((-73.972099 40.765518, -73.971648 40....",119185,533466,4/1/16,New York,New York (Manhattan),Second Avenue,0,1,40.7644,-73.9616
10092,10065,10065,B5,G6350,S,984659,0,+40.7646284,-073.9631438,"POLYGON ((-73.972099 40.765518, -73.971648 40....",140676,628630,8/13/16,New York,New York (Manhattan),2nd Avenue,0,1,40.7644,-73.9616
10092,10065,10065,B5,G6350,S,984659,0,+40.7646284,-073.9631438,"POLYGON ((-73.972099 40.765518, -73.971648 40....",102985,465284,12/13/15,New York,New York (Manhattan),2nd Avenue,0,1,40.7646,-73.9614


## reduce columns

In [97]:
guns_NYC_zip = guns_NYC_zip[['geometry', 'ZCTA5CE10', 'n_killed']]

In [98]:
guns_NYC_zip.head()

,geometry,ZCTA5CE10,n_killed
10092,"POLYGON ((-73.972099 40.765518, -73.971648 40....",10065,0
10092,"POLYGON ((-73.972099 40.765518, -73.971648 40....",10065,0
10092,"POLYGON ((-73.972099 40.765518, -73.971648 40....",10065,0
10092,"POLYGON ((-73.972099 40.765518, -73.971648 40....",10065,0
10092,"POLYGON ((-73.972099 40.765518, -73.971648 40....",10065,0


## groupby number killed and zipcode

In [99]:
n = guns_NYC_zip.groupby(['ZCTA5CE10'])['n_killed'].sum().reset_index()


In [100]:
n.head()

,ZCTA5CE10,n_killed
0,10001,2
1,10002,0
2,10003,1
3,10004,0
4,10005,0


In [101]:
n.rename(columns={'n_killed': 'n_killed_total'}, inplace=True)

In [102]:
guns_NYC_zip.drop(['n_killed'], axis =1, inplace =True)

## merge groupby object with dataset again

In [103]:
guns_NYC_zip = guns_NYC_zip.merge(n, on = 'ZCTA5CE10')

In [105]:
guns_NYC_zip.head()

,geometry,ZCTA5CE10,n_killed_total
0,"POLYGON ((-73.972099 40.765518, -73.971648 40....",10065,0
1,"POLYGON ((-73.972099 40.765518, -73.971648 40....",10065,0
2,"POLYGON ((-73.972099 40.765518, -73.971648 40....",10065,0
3,"POLYGON ((-73.972099 40.765518, -73.971648 40....",10065,0
4,"POLYGON ((-73.972099 40.765518, -73.971648 40....",10065,0


## drop duplicates

In [108]:
guns_NYC_zip.drop_duplicates(subset = 'ZCTA5CE10', keep='first', inplace=True)

In [109]:
guns_NYC_zip.head()

,geometry,ZCTA5CE10,n_killed_total
0,"POLYGON ((-73.972099 40.765518, -73.971648 40....",10065,0
7,"POLYGON ((-73.995969 40.774012, -73.9939739999...",10069,0
8,"POLYGON ((-73.964687 40.77567699999999, -73.96...",10075,1
12,"(POLYGON ((-73.939508 40.779224, -73.939572 40...",10128,0
19,"POLYGON ((-74.01814399999999 40.714291, -74.01...",10282,0


## save as shapefile

In [110]:
guns_NYC_zip.to_file('guns_NY.shp')